In [1]:
import os

requisite='boto3'
os.system(f"pip install {requisite}")
aws_access_key_id = ""
aws_secret_access_key = ""

0

In [1]:
import boto3
import io
import pandas as pd

# Set Buffer
buffer_pbp = io.BytesIO()
buffer_players = io.BytesIO()

# Create connection to S3
s3 = boto3.resource('s3', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)

# Read PBP Data from S3
pbp_object = s3.Object('utd-hackathon', 'event_pbp.parquet')
pbp_object.download_fileobj(buffer_pbp)

df_pbp = pd.read_parquet(buffer_pbp).copy()

print(df_pbp.head())

# Read Players Data from S3
players_object = s3.Object('utd-hackathon', 'game_players.parquet')
players_object.download_fileobj(buffer_players)

df_players = pd.read_parquet(buffer_players).copy()

                      id              gameId   nbaGameId        date  season   
0   2022-10-18-LAL-GSW_2  2022-10-18-LAL-GSW  0022200002  2022-10-18    2023  \
1   2022-10-18-LAL-GSW_4  2022-10-18-LAL-GSW  0022200002  2022-10-18    2023   
2   2022-10-18-LAL-GSW_7  2022-10-18-LAL-GSW  0022200002  2022-10-18    2023   
3   2022-10-18-LAL-GSW_9  2022-10-18-LAL-GSW  0022200002  2022-10-18    2023   
4  2022-10-18-LAL-GSW_10  2022-10-18-LAL-GSW  0022200002  2022-10-18    2023   

  seasonType   nbaTeamId team opponent   offTeamId  ... locY  pts  pbpOrder   
0    Regular        None  LAL      GSW           0  ...  -80    0     20000  \
1    Regular  1610612747  GSW      LAL  1610612747  ...  -80    0     60000   
2    Regular  1610612744  GSW      LAL  1610612747  ...   26    0     70000   
3    Regular  1610612747  LAL      GSW  1610612747  ...   -1    2     90000   
4    Regular  1610612744  GSW      LAL  1610612744  ...   81    0    100000   

  lastName1 statCategory1  lastName2  statCa

In [2]:
import time
from calendar import timegm
def pbpWallClock2Epoch(wallClock):
    utc_time = time.strptime(wallClock, "%Y-%m-%d %H:%M:%SZ")
    return timegm(utc_time)
for i in range(len(df_pbp["wallClock"])):
    df_pbp.at[i, 'wallClock'] = pbpWallClock2Epoch(df_pbp["wallClock"][i])
df_pbp["wallClock"]

0         1666147229
1         1666147234
2         1666147246
3         1666147264
4         1666147280
             ...    
587743    1681078039
587744    1681078081
587745    1681078106
587746    1681078131
587747    1681078154
Name: wallClock, Length: 587748, dtype: object

In [3]:
def excelTime2Epoch(excel_time):
    excel_time = excel_time + ":00"
    utc_excel = time.strptime(excel_time, "%m/%d/%Y %H:%M:%S")
    epoch_excel = timegm(utc_excel)
    return epoch_excel
def closestEvents(epoch_excel, timerange=5*60): # excel format 4/13/2023  11:08:17 PM
    hate = epoch_excel-timerange
    bruh = df_pbp[df_pbp["wallClock"] <= epoch_excel]
    return bruh[bruh["wallClock"] >= hate]
    return [i for i in range(len(df_pbp["wallClock"])) \
            if df_pbp["wallClock"][i] <= epoch_excel and \
            epoch_excel - df_pbp["wallClock"][i] <= timerange]
    return [i for i in range(len(df_pbp["wallClock"])) \
            if pbpWallClock2Epoch(df_pbp["wallClock"][i]) <= epoch_excel and \
            epoch_excel - pbpWallClock2Epoch(df_pbp["wallClock"][i]) <= timerange]

closestEvents(excelTime2Epoch("10/19/2022 02:45"))

,id,gameId,nbaGameId,date,season,seasonType,nbaTeamId,team,opponent,offTeamId,...,locY,pts,pbpOrder,lastName1,statCategory1,lastName2,statCategory2,lastName3,gameStatus,changeDate
0,2022-10-18-LAL-GSW_2,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,None,LAL,GSW,0,...,-80,0,20000,None,None,None,None,None,3,2022-10-20 12:13:07
1,2022-10-18-LAL-GSW_4,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612747,GSW,LAL,1610612747,...,-80,0,60000,None,None,Looney,None,None,3,2022-10-20 12:13:07
2,2022-10-18-LAL-GSW_7,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612744,GSW,LAL,1610612747,...,26,0,70000,Looney,PF,Davis,None,Brothers,3,2022-10-20 12:13:07
3,2022-10-18-LAL-GSW_9,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612747,LAL,GSW,1610612747,...,-1,2,90000,Davis,PTS,None,None,None,3,2022-10-20 12:13:07
4,2022-10-18-LAL-GSW_10,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612744,GSW,LAL,1610612744,...,81,0,100000,Wiggins,None,None,None,None,3,2022-10-20 12:13:07
5,2022-10-18-LAL-GSW_11,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612747,LAL,GSW,1610612744,...,81,0,110000,Westbrook,REB,None,None,None,3,2022-10-20 12:13:07
6,2022-10-18-LAL-GSW_12,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612747,LAL,GSW,1610612747,...,16,0,120000,Beverley,None,None,None,None,3,2022-10-20 12:13:07
7,2022-10-18-LAL-GSW_13,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612744,GSW,LAL,1610612747,...,16,0,130000,Green,REB,None,None,None,3,2022-10-20 12:13:07
8,2022-10-18-LAL-GSW_15,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,None,LAL,GSW,1610612744,...,-80,0,140000,None,None,None,None,None,3,2022-10-20 12:13:07
9,2022-10-18-LAL-GSW_16,2022-10-18-LAL-GSW,0022200002,2022-10-18,2023,Regular,1610612747,LAL,GSW,1610612744,...,189,0,150000,Beverley,PF,Thompson,None,Twardoski,3,2022-10-20 12:13:07


In [4]:
df_twitter = pd.read_csv("mavs_twitter_2223.csv")
for i in range(len(df_twitter["Created"])):
    df_twitter.at[i, "Created"] = excelTime2Epoch(df_twitter["Created"][i])
df_twitter.keys()

Index(['Source', 'SubType', 'ID', 'AuthorUserName', 'AuthorDisplayName',
       'AuthorId', 'Created', 'Message', 'ContentType', 'PhotoUrl',
       'ContextUrl', 'Links', 'AuthorFollowerCount',
       'PostImpressions(Organic)', 'PostProjectedImpressions', 'SocialValue',
       'PostReach', 'VideoViews', 'TotalPublicEngagements', 'LikeCount',
       'Comments', 'Shares/Retweets', 'TotalEngagementsAndInteractions',
       'TotalInteractions', 'EngagementRate', 'FollowerInteractionRate',
       'EngagementAndInteractionRate', 'TWAuthorIsVerified', 'TWRetweetCount',
       'TWReplyCount', 'TWEngagement', 'TWQuoteTweets',
       'TWEngagementAndInteractions', 'TWAuthorCreated', 'FBPageLikeCount',
       'FBCommentCount', 'FBShareCount', 'FBLikeCount', 'FBAngryCount',
       'FBHahaCount', 'FBLoveCount', 'FBSadCount', 'FBWowCount',
       'FBPaidImpressions', 'FBPaidReach', 'FBOrganicImpressions',
       'FBOrganicReach', 'FBPostClicks', 'FBEngagement',
       'FBEngagementAndInteractions',

In [154]:
games = []
def getGamePbp(gameId):
    return df_pbp[df_pbp["gameId"] == game]

for game in set(df_pbp["gameId"]):
    if "DAL" not in game or "PHX" not in game: continue
    df_game = getGamePbp(game)
    games.append((df_game, (min(df_game["wallClock"]), max(df_game["wallClock"]))))

In [155]:
games

[(                          id              gameId   nbaGameId        date   
  2513    2022-10-19-DAL-PHX_2  2022-10-19-DAL-PHX  0022200013  2022-10-19  \
  2514    2022-10-19-DAL-PHX_4  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2515   2022-10-19-DAL-PHX_10  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2516    2022-10-19-DAL-PHX_8  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2517    2022-10-19-DAL-PHX_9  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  ...                      ...                 ...         ...         ...   
  2980  2022-10-19-DAL-PHX_670  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2981  2022-10-19-DAL-PHX_672  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2982  2022-10-19-DAL-PHX_673  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2983  2022-10-19-DAL-PHX_674  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  2984  2022-10-19-DAL-PHX_675  2022-10-19-DAL-PHX  0022200013  2022-10-19   
  
        season seasonType   nbaTeamId team opponent   offTeam

In [166]:
substitutions = {"Rebound": "jumps up and grabs the basketball",
                 "Jump shot": "Shoots a jumpshot",
                 "Dunk": "dunks the basketball",
                 "Alley-Oop": "catches a mid-air pass and dunks the basketball",
                 "Blocks": "blocks the basketball in the air",
                 "Steal": "steals the basketball",
                 "Assist": "throwing the basketball to a teammate"}
def getTweetsDuringTime(minTime, maxTime):
    df_game_tweets = df_twitter[df_twitter["Created"] >= minTime]
    df_game_tweets = df_game_tweets[df_game_tweets["Created"] <= maxTime]
    return df_game_tweets

def getBestTweets(df_tweets):
    df_best = df_tweets.sort_values("LikeCount")
    if len(df_best["LikeCount"]) <= 5: return df_best
    likeAmnt = sorted([i for i in df_best["LikeCount"]])[-5]
    return df_best[df_best["LikeCount"] >= likeAmnt]
tweet2pbp = {}
for game in games[2:3]:
    df_game, (minTime, maxTime) = game
    if len(df_game["gameId"])==0: continue
    gameId = list(df_game["gameId"])[0]
    df_game_tweets = getTweetsDuringTime(minTime, maxTime)
    df_game_tweets = getBestTweets(df_game_tweets)
    messages = []
    events = []
    for message in df_game_tweets["Message"]:
        messages.append(message)
    for time in df_game_tweets["Created"]:
        gameEvents = closestEvents(time)
        gameEvents = gameEvents[gameEvents["gameId"] == gameId]
        events.append(gameEvents["description"])
    for i in range(len(messages)):
        tweet2pbp[messages[i]] = []
        for desc in events[i]:
            if "DAL" not in desc: continue
            proc_desc = desc.lower()
            for prev,sub in substitutions.items():
                proc_desc = proc_desc.replace(prev.lower(),sub.lower())
            tweet2pbp[messages[i]].append(proc_desc)
tweet2pbp.keys()

dict_keys(['Another 📐 https://t.co/Qa7hl9GIBS', "He's like that 🪄 https://t.co/t1KYTaMdq3", '24 minutes to play.\r\n\r\n@Lexus | #MFFL https://t.co/UZv0OOXJzF', 'LD out here playing bully ball 🤯 https://t.co/BVdnaUcjhl', 'Crafty Kai. Count that. https://t.co/O0HebGounZ'])

In [167]:
key = 'LD out here playing bully ball 🤯 https://t.co/BVdnaUcjhl'
print(key)
for i in tweet2pbp[key]:
    print(i)

LD out here playing bully ball 🤯 https://t.co/BVdnaUcjhl
[dal 91-83] doncic driving floating shoots a jumpshot: made (23 pts)
[dal] holiday foul: shooting (2 pf) (2 fta) (e lewis)
[dal] doncic 3pt shot: missed


In [168]:
respell_input = [key] + tweet2pbp[key]
respell_input

['LD out here playing bully ball 🤯 https://t.co/BVdnaUcjhl',
 '[dal 91-83] doncic driving floating shoots a jumpshot: made (23 pts)',
 '[dal] holiday foul: shooting (2 pf) (2 fta) (e lewis)',
 '[dal] doncic 3pt shot: missed']

In [169]:
import requests
import json

respell_authorization = ""

response = requests.post(
  "https://api.respell.ai/v1/run",
  headers={
    # This is your API key
    'Authorization': respell_authorization,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
  },
  data=json.dumps({
    "spellId": "HKyGY9mpqGRfWnISuyvMi",
    # This field can be omitted to run the latest published version
    "spellVersionId": 'Wm8BGoPj7nI3Pw6rJVwkU',
    # Fill in dynamic values for each of your 2 input blocks
    "inputs": {
      "text_input": '\n'.join(respell_input),
      "selection_instructions": 'I will give you the following input. The first line will have a tweet from a Dallas Mavericks account. Each of the lines afterwards will give an event from a Dallas Mavericks Game. Each event occurred at around the same time the tweet was created. I want you to determine which event best corresponds to the tweet. I want the output to be a single line in the following format:\n(name), an NBA player for the Dallas Mavericks, (event).\nFor each name, if only the last name is given, replace this with the first and last name.\nAs an example, if the tweet is mentioning Luka Dončić, then the best corresponding event would involve him.',
    }
  }),
)

In [170]:
clean = json.loads(response.text)
selected_pbps = json.loads(response.text)["outputs"]["selected_pbp"].split("\n")
selected_pbps

['Luka Dončić, an NBA player for the Dallas Mavericks, driving floating shoots a jumpshot: made (23 pts).']

In [127]:
import requests
import json
import time



BASE_URL = "https://api.luan.tools/api/tasks/"
HEADERS = {
    'Authorization': '',
    'Content-Type': 'application/json'
}


def send_task_to_dream_api(style_id, prompt, target_img_path=None):
    """
    Send requests to the dream API.
    prompt is the text prompt.
    style_id is which style to use (a mapping of ids to names is in the docs).
    target_img_path is an optional path to an image to influence the generation.
    """

    # Step 1) make a POST request to https://api.luan.tools/api/tasks/
    post_payload = json.dumps({
        "use_target_image": bool(target_img_path)
    })
    post_response = requests.request(
        "POST", BASE_URL, headers=HEADERS, data=post_payload)
   
    # Step 2) skip this step if you're not sending a target image otherwise,
    # upload the target image to the url provided in the response from the previous POST request.
    if target_img_path:
        target_image_url = post_response.json()["target_image_url"]
        with open(target_img_path, 'rb') as f:
            fields = target_image_url["fields"]
            fields ["file"] = f.read()
            requests.request("POST", url=target_image_url["url"], files=fields)

    # Step 3) make a PUT request to https://api.luan.tools/api/tasks/{task_id}
    # where task id is provided in the response from the request in Step 1.
    print(post_response.json())
    task_id = post_response.json()['id']
    task_id_url = f"{BASE_URL}{task_id}"
    put_payload = json.dumps({
        "input_spec": {
            "style": style_id,
            "prompt": prompt,
            "target_image_weight": 0.1,
            "width": 960,
            "height": 1560
    }})
    requests.request(
        "PUT", task_id_url, headers=HEADERS, data=put_payload)

    # Step 4) Keep polling for images until the generation completes
    while True:
        response_json = requests.request(
            "GET", task_id_url, headers=HEADERS).json()

        state = response_json["state"]

        if state == "completed":
            r = requests.request(
                "GET", response_json["result"])
            with open("image.jpg", "wb") as image_file:
                image_file.write(r.content)
            print("image saved successfully :)")
            break

        elif state =="failed":
            print("generation failed :(")
            break

        time.sleep(3)
   
    # Step 5) Enjoy your beautiful artwork :3
send_task_to_dream_api(style_id = '84', prompt=selected_pbps[0], target_img_path=None)

{'id': 'ca6b9e83-1f30-4f1c-9fa5-650145c86ff6', 'input_spec': None, 'state': 'input', 'photo_url_list': [], 'result': None, 'use_target_image': False, 'target_image_url': None, 'created_at': '2023-04-16T09:43:07.130139+00:00', 'updated_at': '2023-04-16T09:43:07.130139+00:00', 'model': None, 'is_nsfw': False}
image saved successfully :)


In [128]:
selected_pbps[0]

'Luka Dončić, an NBA player for the Dallas Mavericks, threw the basketball to a teammate: Wood (8 ast).'